## Plant disease classification model using TensorFlow and Transfer Learning

Please download the dataset from the below url

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3,5)

# Common imports
import numpy as np
import os
import glob

try:
    # %tensorflow_version only exists in Colab
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False
    
# Tensorflow >= 2.0 is required
import tensorflow as tf
assert tf.__version__ >= '2.0'

if IS_COLAB:
    print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    
if tf.config.list_physical_devices('GPU'):
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
        
    config = ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.2
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)
else:
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize = 12)
mpl.rc('ytick', labelsize = 12)


## Getting the data

The dataset can be download from the following link: 

https://www.kaggle.com/vipoooool/new-plant-diseases-dataset

In [ ]:
# import the libraries as shown below
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet152V2 import ResNet152V2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [256,256]

train_path = os.path.join(data_path, 'train')
valid_path = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'


In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
resnet152V2 =tf.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [ ]:
# don't train existing weights
for layer in resnet152V2.layers:
    layer.trainable = False

In [ ]:
  # useful for getting number of output classes
folders = glob('../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/*')

In [ ]:
# Modifying the output layer
x = Flatten()(resnet152V2.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# Create a model object
model = Model(inputs=resnet152V2.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.008)
model.compile(
  loss=['categorical_crossentropy'],
  optimizer=optimizer,
  metrics=['accuracy']
)


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (256, 256),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (256, 256),
                                            batch_size = 128,
                                            class_mode = 'categorical')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience=1)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('model_resnet152V2.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
y_pred_max = np.argmax(y_pred, axis=1)

In [ ]:
y_pred_max

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_loaded = load_model('model_resnet50.h5')